In [2]:
import numpy as np

# Load preprocessed image data and dummy labels
X_data = np.load("../data/X_anime.npy")
y_data = np.load("../data/y_anime.npy")

# Check data shape and values
print("✅ X_data shape:", X_data.shape)
print("✅ y_data shape:", y_data.shape)
print("✅ First 5 labels:", y_data[:5])

✅ X_data shape: (120, 512, 512, 3)
✅ y_data shape: (120,)
✅ First 5 labels: [0 0 0 0 0]


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 1️⃣ Build the CNN model
model = Sequential()

# 2️⃣ First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3️⃣ Second conv block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4️⃣ Flatten and Dense
model.add(Flatten())
model.add(Dense(128, activation='relu'))

# 5️⃣ Output layer - 2 dummy classes
model.add(Dense(2, activation='softmax'))

# 6️⃣ Show summary
model.summary()

c:\AI\anime-face-classifier\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1016064)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   130,056,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130,075,970 (496.20 MB)

 Trainable params: 130,075,970 (496.20 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# 1. Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # because y_data = integers like 0
    metrics=['accuracy']
)

# 2. Train the model
history = model.fit(
    X_data,           # input data (images)
    y_data,           # dummy labels (all 0s for now)
    epochs=5,         # full passes through data (can increase later)
    batch_size=8,     # 8 images per step
    validation_split=0.2,  # 20% of data used for validation
    verbose=1
)

Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8162 - loss: 0.1775 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [5]:
# Save the model
model.save("models/anime_cnn_model.h5")

print("✅ Model saved successfully!")

✅ Model saved successfully!


In [ ]:
#after reisize_images.ipynb


Retrain CNN with Real Labels & Resized Input (Model v2)

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load resized images (128x128) and real binary labels
X_data = np.load("../data/X_anime_128.npy")
y_data = np.load("../data/y_anime_2class.npy")

# Normalize
X_data = X_data / 255.0

# Build CNN (Version 2)
model = Sequential()

# Convolutional Block 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Block 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dense Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Helps avoid overfitting
model.add(Dense(2, activation='softmax'))  # Binary output

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_data,
    y_data,
    epochs=10,
    batch_size=8,
    validation_split=0.2,
    verbose=1
)

# Save version 2 of the model
model.save("../models/anime_cnn_model_v2.h5")
print("✅ Model v2 saved successfully!")

c:\AI\anime-face-classifier\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.5583 - loss: 2.4004 - val_accuracy: 0.0000e+00 - val_loss: 1.1544
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.6541 - loss: 0.6220 - val_accuracy: 0.0000e+00 - val_loss: 0.8989
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.6386 - loss: 0.6494 - val_accuracy: 0.0417 - val_loss: 0.7666
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.7832 - loss: 0.6016 - val_accuracy: 0.0833 - val_loss: 0.8706
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8584 - loss: 0.4984 - val_accuracy: 0.3333 - val_loss: 0.9012
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.7762 - loss: 0.3966 - val_accuracy: 0.3333 - val_loss: 1.0952
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.9342 - loss: 0.2949 - val_accuracy: 0.2917 - val_loss: 1.9011
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.9495 - loss: 0.1869 - val_accuracy: 0.

✅ Model v2 saved successfully!
